In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import random
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
import tensorflow as tf
cocomo=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomo81_dataset.csv",header=None)
columns_cocomo=['rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','actual']
cocomo.set_axis(columns_cocomo,axis='columns',inplace=True)
cocomo.set_axis(range(1,64),axis=0 ,inplace=True)
cocomo.rename_axis("Features", axis=1,inplace=True)
cocomo.rename_axis("Projects", axis=0,inplace=True)

In [2]:


# Define your dataset (X) and corresponding effort values (y)
# Replace X and y with your actual dataset

# Feature matrix X should have shape (n_samples, n_features)
X = cocomo.drop('actual',axis=1).values
# Target values y should have shape (n_samples,)
y = cocomo['actual'].values

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)

#=========================================================================================================================
# Define the fitness function
def fitness_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    cross_val_rmse = []
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=0)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)
#========================================================================================================================

max_fitness_score=0
max_score_features=0

# Genetic Algorithm Parameters
population_size = 10
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.1

# Create an initial population of feature subsets
population = []
for _ in range(population_size):
    subset = [random.randint(0, 1) for _ in range(X.shape[1])]
    population.append(subset)
    
# Genetic Algorithm
for generation in range(num_generations):
    # Evaluate fitness for each feature subset in the population
    fitness_scores = [fitness_function(subset) for subset in population]
    if max(fitness_scores) > max_fitness_score:
        max_fitness_score = max(fitness_scores)
        max_score_features =population[fitness_scores.index(max(fitness_scores))]
    
    # Selection
    selected_population = sorted(population, key=lambda x: fitness_function(x), reverse=True)
    
    # Crossover
    elitism_num = 2
    offspring_population = selected_population[:elitism_num] # Elitism, preserving the top individuals
    for i in range(0, population_size, 2):
        parent1 = selected_population[i]
        parent2 = selected_population[i + 1]
        
        if random.random() < crossover_rate:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
        else:
            child1 = parent1
            child2 = parent2
        
        offspring_population.append(child1)
        offspring_population.append(child2)
    
    # Mutation
    for i in range(2,population_size):
        if random.random() < mutation_rate:
            mutated_feature = random.randint(0, X.shape[1] - 1)
            offspring_population[i][mutated_feature] = 1 - offspring_population[i][mutated_feature]
    
    # Replace the old population with the new offspring population
    population = offspring_population

# Select the best feature subset from the final population
best_subset = max(population, key=fitness_function)
selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), best_subset) if is_selected]

print(selected_features)
#======================================================================================================================    
    
# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
        'neurons_input': (10, 50),
        'neurons_hidden': (10, 50),
        'num_hidden_layers': (1, 5),
        'learning_rate': (1e-5, 1e-2),
        'batch_size': (8, 32),
        'epochs': (5,20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)
    
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")
#===================================================================================================================

# train the model using max_features

# X_selected = X[:, max_score_features]
# X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# model3 = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)),
#     tf.keras.layers.Dense(units=32, activation='relu'),
#     tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
# ])

#  # Step 4: Compile the model.
# model3.compile(optimizer='adam', loss='mean_squared_error')

#  # Step 5: Train the model.
# model3.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1)

# # You can now use the trained model for prediction or further evaluation
# y_pred = model3.predict(X_test_scaled)

# # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
# rmse3 = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f"Root Mean Squared Error (RMSE): {rmse3}")

# # Convert y_pred and y_test to NumPy arrays
# y_pred = np.array(y_pred).flatten()
# y_test = np.array(y_test)

# # Calculate Mean Magnitude of Relative Error (MMRE)
# mmre = np.mean(np.abs((y_test - y_pred) / y_test))
# print(f"Mean Magnitude of Relative Error (MMRE): {mmre:.2f}")




# print(best_subset," \n ", max_score_features)




1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 34ms/step


...

1/1 [==============================] - 0s 41ms/step
| 3         | -3.629e+0 | 27.98     | 8.185     | 0.001826  | 17.34     | 22.17     | 3.099     |
1/1 [==============================] - 0s 39ms/step
| 4         | -3.626e+0 | 18.37     | 9.368     | 0.006122  | 15.58     | 21.69     | 2.465     |
1/1 [==============================] - 0s 36ms/step
| 5         | -3.626e+0 | 18.95     | 16.78     | 0.002005  | 30.57     | 33.7      | 1.186     |
1/1 [==============================] - 0s 52ms/step
| 6         | -3.629e+0 | 22.58     | 7.558     | 0.0006599 | 47.96     | 48.63     | 4.234     |
1/1 [==============================] - 0s 38ms/step
| 7         | -3.62e+03 | 15.31     | 6.465     | 0.006845  | 27.61     | 14.88     | 2.981     |
1/1 [==============================] - 0s 41ms/step
| 8         | -3.21e+03 | 8.825     | 18.64     | 0.002595  | 36.5      | 22.47     | 3.08      |
1/1 [==============================] - 0s 47ms/step
| 9         | -3.293e+0 | 21.12     | 7.773     

1/1 [==============================] - 0s 49ms/step
|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 36ms/step
| 1         | -301.1    | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/1 [==============================] - 0s 47ms/step
| 2         | -1.261e+0 | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/1 [==============================] - 0s 40ms/step
| 3         | -424.2    | 27.98     | 8.185     | 0.001826  | 17.34     | 22.17     | 3.099     |
1/1 [==============================] - 0s 37ms/step
| 4         | -418.9    | 18.37     | 9.368     | 0.006122  | 15.58     | 21.69     | 2.465     |
1/1 [==============================] - 0s 35ms/step
| 5         | -421.1    | 18.95     | 16.78     | 0.002005  | 30.57     | 33.7      | 1.186     |
1/

1/1 [==============================] - 0s 49ms/step
| 38        | -425.0    | 30.07     | 5.715     | 1e-05     | 32.54     | 23.18     | 5.0       |
1/1 [==============================] - 0s 35ms/step
| 39        | -402.9    | 31.5      | 17.8      | 0.007232  | 21.17     | 27.91     | 1.503     |
1/1 [==============================] - 0s 35ms/step
| 40        | -425.1    | 24.87     | 10.52     | 1e-05     | 13.92     | 10.0      | 1.0       |
1/1 [==============================] - 0s 44ms/step
|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 35ms/step
| 1         | -565.1    | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/1 [==============================] - 0s 47ms/step
| 2         | -657.8    | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/

1/1 [==============================] - 0s 46ms/step
| 35        | -512.8    | 21.21     | 8.096     | 0.006959  | 41.47     | 46.71     | 4.263     |
1/1 [==============================] - 0s 37ms/step
| 36        | -494.3    | 10.07     | 10.92     | 0.01      | 37.16     | 32.33     | 2.663     |
1/1 [==============================] - 0s 41ms/step
| 37        | -544.1    | 26.49     | 12.0      | 0.008819  | 28.51     | 27.07     | 3.279     |
1/1 [==============================] - 0s 42ms/step
| 38        | -643.8    | 26.22     | 13.34     | 0.002955  | 27.46     | 27.29     | 3.381     |
1/1 [==============================] - 0s 44ms/step
| 39        | -441.3    | 9.356     | 9.681     | 0.00653   | 36.96     | 32.45     | 2.2       |
1/1 [==============================] - 0s 47ms/step
| 40        | -560.2    | 20.49     | 8.947     | 0.008485  | 40.87     | 47.63     | 4.058     |
1/1 [==============================] - 0s 37ms/step
|   iter    |  target   | batch_... |  epochs   

1/1 [==============================] - 0s 38ms/step
| 32        | -1.041e+0 | 8.0       | 18.23     | 0.005452  | 37.35     | 22.33     | 2.721     |
1/1 [==============================] - 0s 43ms/step
| 33        | -1.255e+0 | 9.62      | 15.13     | 0.003579  | 40.26     | 11.66     | 3.805     |
1/1 [==============================] - 0s 43ms/step
| 34        | -562.9    | 11.28     | 16.18     | 0.009791  | 14.47     | 34.83     | 3.917     |
1/1 [==============================] - 0s 34ms/step
| 35        | -2.05e+03 | 8.021     | 17.41     | 0.002062  | 36.61     | 20.78     | 1.872     |
1/1 [==============================] - 0s 46ms/step
| 36        | -1.268e+0 | 11.14     | 15.94     | 0.008361  | 15.37     | 35.04     | 4.104     |
1/1 [==============================] - 0s 43ms/step
| 37        | -1.28e+03 | 8.934     | 17.33     | 0.009155  | 40.6      | 10.58     | 3.78      |
1/1 [==============================] - 0s 45ms/step
| 38        | -813.3    | 10.17     | 17.21     